In [30]:
# import necessary Libraries 
import pandas as pd

In [31]:
! pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [17]:
! pip show kaggle

Name: kaggle
Version: 1.6.14
Summary: Kaggle API
Home-page: https://github.com/Kaggle/kaggle-api
Author: Kaggle
Author-email: support@kaggle.com
License: Apache 2.0
Location: c:\users\danda\appdata\roaming\python\python39\site-packages
Requires: tqdm, python-slugify, certifi, six, python-dateutil, bleach, urllib3, requests
Required-by: 


In [24]:
import os
os.environ['kaggle']="c:\\users\\danda\\appdata\\roaming\\python\\python39\\site-packages\\kaggle"

In [27]:
kaggle_dir

'C:\\Users\\danda\\.kaggle'

In [28]:
import os
import platform
import shutil

# Specify the folder where the script should run
base_folder = 'D:\era_v2\ERA_V2\S20'

# Change current working directory to the base folder
os.chdir(base_folder)

# Create directory for storing Kaggle API key
kaggle_dir = os.path.expanduser('~/.kaggle')  # Default location on Unix-like systems

# Check platform and adjust directory path accordingly
if platform.system() == 'Windows':
    kaggle_dir = os.path.join(base_folder, '.kaggle')
elif platform.system() == 'Darwin':  # macOS
    kaggle_dir = os.path.join(os.path.expanduser('~'), '.kaggle')

# Create the directory if it doesn't exist
os.makedirs(kaggle_dir, exist_ok=True)

# Copy the Kaggle API key file
try:
    shutil.copy('kaggle.json', os.path.join(kaggle_dir, 'kaggle.json'))
except AttributeError:  # Handle Windows
    shutil.copy('kaggle.json', os.path.join(kaggle_dir, 'kaggle.json'))

# Set appropriate permissions for the Kaggle API key file
try:
    os.system('chmod 600 {}'.format(os.path.join(kaggle_dir, 'kaggle.json')))
except AttributeError:  # Handle Windows
    # On Windows, no need to change permissions
    pass

# Create directory for dataset download
os.makedirs(os.path.join(base_folder, 'corpus'), exist_ok=True)

# Download dataset using Kaggle CLI
try:
    os.system('kaggle datasets download -d furcifer/bangla-newspaper-dataset -p corpus')
except FileNotFoundError:
    print('Kaggle CLI not found. Please install Kaggle API: https://github.com/Kaggle/kaggle-api')


In [29]:
! kaggle datasets download -d furcifer/bangla-newspaper-dataset -p corpus

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# ! mkdir corpus
# ! kaggle datasets download -d furcifer/bangla-newspaper-dataset
# !kaggle datasets download -d disisbig/hindi-wikipedia-articles-172k
# !unzip /content/hindi-wikipedia-articles-172k.zip -d /content/corpus

A subdirectory or file corpus already exists.


'kaggle' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
df=pd.read_json("/Users/kvzm411/Desktop/ERA V2/ERA_V2/S20/data.json")

In [4]:
df.head(2)

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content
0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,অনলাইন ডেস্ক,sports,খেলা,"০৪ জুলাই ২০১৩, ২৩:০৯","০৪ জুলাই ২০১৩, ২৩:১১",[টেনিস],0,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...


In [6]:
df = df[['content']]


In [7]:
df.head(1)

,content
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...


In [8]:
# Combine the content of the column into a single variable, separated by newline character
combined_content = '\n'.join(df['content'].astype(str).tolist())


In [9]:
combined_content[0:100]

'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার '

In [10]:
print("length of dataset1 in characters: ", len(combined_content))

length of dataset1 in characters:  820065032


In [14]:
text=combined_content

In [15]:

vocab = sorted(list(set(text)))
print('Length of Vocabulary:',len(vocab))

Length of Vocabulary: 572


In [16]:
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print("length of text:", len(text))
print("length of tokens:", len(tokens))

length of text: 820065032
length of tokens: 2184630568


In [17]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [18]:

stats = get_stats(tokens)

In [19]:
list(stats.items())[:5]

[((224, 166), 519500466),
 ((166, 151), 8256353),
 ((151, 224), 7652319),
 ((166, 190), 71219243),
 ((190, 224), 59313647)]

In [20]:
top_pair = max(stats, key=stats.get)
top_pair

(224, 166)

In [21]:
chr(224), chr(166)

('à', '¦')

In [22]:
def merge(ids, pair, idx):
  i = 0
  newids = []
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  
  return newids

In [23]:
num_merges = len(vocab) - 256
num_merges

316

In [24]:
ids = list(tokens)
merges = {}
for i in range(30):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print('Merges completed')

In [ ]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

In [ ]:
import pickle
with open('merges.pkl', 'wb') as file:
    pickle.dump(merges, file)
with open('vocab.pkl', 'wb') as file:
    pickle.dump(vocab, file)

In [ ]:
import string, re

tokenizer_hindi_pattern=re.compile(r'(['+string.punctuation+r'\u0964\u0965\uAAF1\uAAF0\uABEB\uABEC\uABED\uABEE\uABEF\u1C7E\u1C7F'+r'])')
pattern_num_seq=re.compile(r'([0-9]+ [,.:/] )+[0-9]+')

def tokenize_hindi_text(text): 
    """tokenize string for Indian language scripts using Brahmi-derived scripts

    A trivial tokenizer which just tokenizes on the punctuation boundaries. 
    This also includes punctuations for the Indian language scripts (the 
    purna virama and the deergha virama). This is a language independent 
    tokenizer

    Args:
        text (str): text to tokenize

    Returns:
        list: list of tokens

    """
    tok_str=tokenizer_hindi_pattern.sub(r' \1 ',text.replace('\t',' '))
    s=re.sub(r'[ ]+',' ',tok_str).strip(' ')
    
    # do not tokenize numbers and dates
    new_s=''
    prev=0
    for m in pattern_num_seq.finditer(s):
        start=m.start()
        end=m.end()
        if start>prev:
            new_s=new_s+s[prev:start]
            new_s=new_s+s[start:end].replace(' ','')
            prev=end
   
    new_s=new_s+s[prev:]
    s=new_s
    
    return s.split(' ')

In [ ]:
tokenize_hindi_text('সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি')